In [ ]:
from tensorflow.config import list_physical_devices
from tensorflow.keras import mixed_precision
from tensorflow.config.experimental import set_memory_growth

### Variables set up

In [ ]:
MAX_LEN = 512
NUM_HEAD = 8
FF_DIM = 1024
NUM_LAYERS = 8
EMBED_DIM = 256

BATCH_SIZE = 256
LR = 0.0004

### GPU set up

In [ ]:
gpus = list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        set_memory_growth(gpu, True)
mixed_precision.set_global_policy('mixed_float16')